In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from func import *
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM

/home/mainak/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/home/mainak/.local/lib/python2.7/site-packages/pyneb/utils/manage_atomic_data.py:528: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  gsconf = np.genfromtxt(execution_path('../atomic_data/levels/gsconfs.dat'), names=['atom', 'gsconf'], dtype=None)


In [2]:
def vel_sigma_wing_err(obj):
    hdu = fits.open('%s_central_fit.fits'%(obj))
    central_tab = hdu[1].data
    central_columns = hdu[1].header
    wing_err = central_tab.field('vel_sigma_OIII_br_err')[0]
    return wing_err

def flux_density_wing_moffat(OIII_br_data):
    flux_5007 = np.sum(OIII_br_data)
    return flux_5007

def flux_density_wing_moffat_err(OIII_br_err):
    err_squared = [x**2 for x in OIII_br_err]
    total_err = np.sqrt(np.sum(err_squared))
    return total_err

def flux_density_and_err_wing(obj):
    hdu = fits.open('%s_spectro_central_fit.fits'%(obj))
    central_tab = hdu[1].data
    central_columns = hdu[1].header
    amp_wing = central_tab.field('amp_OIII5007_br')[0]
    amp_wing_err = central_tab.field('amp_OIII5007_br_err')[0]
    flux_den_par = [amp_wing,amp_wing_err]  
    return flux_den_par

def flux_OIII_br_moffat(OIII_br_data,sigma_spectro_OIII_5007_br,sigma_spectro_OIII_4959_br):
    flux_5007 = np.sum(OIII_br_data)*(np.sqrt(2.*np.pi)*np.fabs(sigma_spectro_OIII_5007_br))
    #flux_4959 = 0.33*np.sum(OIII_br_data)*(np.sqrt(2.*np.pi)*np.fabs(sigma_spectro_OIII_4959_br)) 
    flux = flux_5007*(10**(-16))  #+ flux_4959)*(10**(-16))  
    return flux

def flux_OIII_br_spectro(amp_OIII5007_br_spec,sigma_spectro_OIII_5007_br,sigma_spectro_OIII_4959_br):
    flux_5007 = amp_OIII5007_br_spec*(np.sqrt(2.*np.pi)*np.fabs(sigma_spectro_OIII_5007_br))
    #flux_4959 = 0.33*amp_OIII5007_br_spec*(np.sqrt(2.*np.pi)*np.fabs(sigma_spectro_OIII_4959_br)) 
    flux = flux_5007*(10**(-16))  #+ flux_4959)*(10**(-16))  
    return flux

def flux_OIII_br_nonspec(amp_OIII5007_br_fit,sigma_nonspec_OIII_5007_br,sigma_nonspec_OIII_4959_br):
    flux_5007 = amp_OIII5007_br_fit*(np.sqrt(2.*np.pi)*np.fabs(sigma_nonspec_OIII_5007_br))
    #flux_4959 = 0.33*amp_OIII5007_br_fit*(np.sqrt(2.*np.pi)*np.fabs(sigma_nonspec_OIII_4959_br)) 
    flux = flux_5007*(10**(-16))  #+ flux_4959)*(10**(-16))  
    return flux

def sigma_err(vel_sigma_OIII_br,vel_sigma_OIII_br_err,sigma_spectro_OIII_br):
    c = 300000
    delta_sigma = ((vel_sigma_OIII_br_err/vel_sigma_OIII_br)/(1-(vel_sigma_OIII_br/c)))*sigma_spectro_OIII_br
    return delta_sigma


def flux_OIII_br_err(flux,flux_density,flux_density_err,sigma_spectro_OIII_5007_br,sigma_spectro_OIII_4959_br,sigma_spectro_OIII_5007_br_err,sigma_spectro_OIII_4959_br_err):
    amp_contribution = flux_density_err/flux_density
    #print amp_contribution
   # sigma_contribution = (sigma_spectro_OIII_5007_br_err + 0.33*sigma_spectro_OIII_4959_br_err)/(sigma_spectro_OIII_5007_br + 0.33*sigma_spectro_OIII_4959_br)
    flux_err_total = flux* amp_contribution 
    return flux_err_total

def lum_OIII_br_moffat(z,flux_wing_moffat):
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    Hb_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux_wing_moffat)
    return Hb_lum

def lum_OIII_br_spectro(z,flux_wing_spectro):
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    Hb_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux_wing_spectro)
    return Hb_lum

def lum_OIII_br_nonspec(z,flux_wing_nonspec):
    cosmo = FlatLambdaCDM(H0=70 * u.km / u.s / u.Mpc, Om0=0.3)
    D_L = cosmo.luminosity_distance(z)
    dist = D_L.to(u.cm)
    Hb_lum = np.log10(4*np.pi)+2*np.log10(dist.value)+np.log10(flux_wing_nonspec)
    return Hb_lum

def par_tab(obj,flux_wing_moffat,flux_wing_spectro,flux_wing_nonspec,lum_wing_moffat,lum_wing_spectro,lum_wing_nonspec,flux_wing_moffat_err,flux_wing_spectro_err):
    par = [flux_wing_moffat,flux_wing_spectro,flux_wing_nonspec,lum_wing_moffat,lum_wing_spectro,lum_wing_nonspec,flux_wing_moffat_err,flux_wing_spectro_err]
    column_names={'F_OIII_br_moffat':0,'F_OIII_br_fixkin':1,'F_OIII_br_nonfixkin':2,'log_L_OIII_br_moffat':3,'log_L_OIII_br_fixkin':4,'log_L_OIII_br_nonfixkin':5,'F_wing_moffat_err':6,'F_wing_spectro_err':7}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[par[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s_compare_flux_table.fits'%(obj),overwrite=True)      
    

In [3]:
def algorithmic_script(obj,z):
    print '%s'%(obj)
    (amp_Hb1,amp_Hb2,vel_Hb1,vel_Hb2,vel_sigma_Hb1,vel_sigma_Hb2,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,vel_offset,m,c) = par(obj)
    (amp_Hb_fit,amp_OIII5007_fit,vel_OIII_fit,vel_sigma_OIII_fit,amp_Hb_br_fit,amp_OIII5007_br_fit,vel_OIII_br_fit,vel_sigma_OIII_br_fit,amp_Hb1_fit,amp_Fe5018_1_fit,vel_Hb1_fit,vel_sigma_Hb1_fit,amp_Hb2_fit,amp_Fe5018_2_fit,vel_Hb2_fit,vel_sigma_Hb2_fit,m_fit,c_fit) = loadtab('%s_non_spectro_central_fit.fits'%(obj))                          
    (amp_Hb_spec,amp_OIII5007_spec,amp_OIII5007_br_spec,amp_Hb_br_spec,amp_Hb1_spec,amp_Hb2_spec,amp_Fe5018_1_spec,amp_Fe5018_2_spec,m_spec,c_spec) = loadspectab('%s_spectro_central_fit.fits'%(obj))  
    (OIII_br_data) = loadwing('maps_%s.fits'%(obj))
    (OIII_br_err) = loadwing_err('subcube_par_err_%s.fits'%(obj))
    
    flux_density_OIII_br_moffat = flux_density_wing_moffat(OIII_br_data)
    flux_density_OIII_br_moffat_err = flux_density_wing_moffat_err(OIII_br_err)
    
   
    sigma_spectro_OIII_5007_br = line_width_recons(vel_sigma_OIII_br,rest_line=5006.8,inst_res_fwhm=0)
    sigma_spectro_OIII_4959_br = line_width_recons(vel_sigma_OIII_br,rest_line=4958.9,inst_res_fwhm=0) 
    
    sigma_nonspec_OIII_5007_br = line_width_recons(vel_sigma_OIII_br_fit,rest_line=5006.8,inst_res_fwhm=0)
    sigma_nonspec_OIII_4959_br = line_width_recons(vel_sigma_OIII_br_fit,rest_line=4958.9,inst_res_fwhm=0)
    
    #print sigma_spectro_OIII_4959_br,sigma_spectro_OIII_5007_br
        
    vel_sigma_OIII_br_err = vel_sigma_wing_err(obj)
    #print vel_sigma_OIII_br, vel_sigma_OIII_br_err
    
    sigma_spectro_OIII_5007_br_err = sigma_err(vel_sigma_OIII_br,vel_sigma_OIII_br_err,sigma_spectro_OIII_5007_br)
    sigma_spectro_OIII_4959_br_err = sigma_err(vel_sigma_OIII_br,vel_sigma_OIII_br_err,sigma_spectro_OIII_4959_br)
    
    #print sigma_spectro_OIII_4959_br_err,sigma_spectro_OIII_5007_br_err
    #print sigma_spectro_OIII_4959_br,sigma_spectro_OIII_4959_br_err
    #print sigma_spectro_OIII_5007_br,sigma_spectro_OIII_5007_br_err
    
    [flux_density_OIII_br_spectro,flux_density_OIII_br_spectro_err] = flux_density_and_err_wing(obj)
    
    #print flux_density_OIII_br_spectro_err/flux_density_OIII_br_spectro
    #print (sigma_spectro_OIII_5007_br_err + 0.33*sigma_spectro_OIII_4959_br_err)/(sigma_spectro_OIII_5007_br + 0.33*sigma_spectro_OIII_4959_br)
    
    flux_wing_moffat = flux_OIII_br_moffat(OIII_br_data,sigma_spectro_OIII_5007_br,sigma_spectro_OIII_4959_br)
    flux_wing_spectro = flux_OIII_br_spectro(amp_OIII5007_br_spec,sigma_spectro_OIII_5007_br,sigma_spectro_OIII_4959_br)
    flux_wing_nonspec = flux_OIII_br_nonspec(amp_OIII5007_br_fit,sigma_nonspec_OIII_5007_br,sigma_nonspec_OIII_4959_br)
    #print flux_wing_moffat,flux_wing_spectro #,flux_wing_nonspec
    a = flux_wing_spectro/flux_wing_moffat
    print a
    
    flux_wing_moffat_err = flux_OIII_br_err(flux_wing_moffat,flux_density_OIII_br_moffat,flux_density_OIII_br_moffat_err,sigma_spectro_OIII_5007_br,sigma_spectro_OIII_4959_br,sigma_spectro_OIII_5007_br_err,sigma_spectro_OIII_4959_br_err)
    flux_wing_spectro_err = flux_OIII_br_err(flux_wing_spectro,flux_density_OIII_br_spectro,flux_density_OIII_br_spectro_err,sigma_spectro_OIII_5007_br,sigma_spectro_OIII_4959_br,sigma_spectro_OIII_5007_br_err,sigma_spectro_OIII_4959_br_err)
 
    
    lum_wing_moffat = lum_OIII_br_moffat(z,flux_wing_moffat)
    lum_wing_spectro = lum_OIII_br_spectro(z,flux_wing_spectro)
    lum_wing_nonspec = lum_OIII_br_nonspec(z,flux_wing_nonspec)
    
   # print lum_wing_moffat,lum_wing_spectro,lum_wing_nonspec 
    
    par_tab(obj,flux_wing_moffat,flux_wing_spectro,flux_wing_nonspec,lum_wing_moffat,lum_wing_spectro,lum_wing_nonspec,flux_wing_moffat_err,flux_wing_spectro_err)

In [4]:
z = {"HE0021-1819":0.053197,"HE0040-1105":0.041692 ,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143,"HE0253-1641":0.031588
    ,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009,"HE0433-1028":0.035550
    ,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986,"HE1029-1831":0.040261
    ,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009,"HE1330-1013":0.022145,"HE1353-1917":0.035021
    ,"HE1417-0909":0.044,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}
objs = z.keys()

for obj in objs:bj
    algorithmic_script(obj,z[obj])  

HE2233+0124
0.9854906043885692
HE0429-0247
1.0263642412954603
HE0232-0900
1.1481939552298366
HE2222-0026
0.9760101065593737
HE0021-1819
1.0598704623157398
HE0853+0102
1.1349716899754214
HE1237-0504
1.0274838623529237
HE1011-0403
1.0337892571623277
HE0040-1105
1.076720425898983
HE0253-1641
0.9720180418066144
HE0119-0118
1.0349552402921165
HE0934+0119
1.0345664469938145
HE1017-0305
1.1882152691543937
HE0345+0056
1.0188112700665166
HE0412-0803
1.1633322893268176
HE0108-4743
1.200759130911763
HE0433-1028
1.0488014755710673
HE1330-1013
0.9371724382316754
HE1417-0909
1.1200559638526506
HE0227-0913
0.9710860666287507
HE1029-1831
1.3192034782092792
HE2302-0857
0.9826002228346972
HE1107-0813
1.0151737686147113
HE0114-0015
1.3396279259949273
HE2211-3903
1.2333766501847887
HE1108-2813
1.238173349133259
HE1126-0407
1.0068131741829471
HE0351+0240
1.3410404543547219
HE0224-2834
1.0449789488946348
HE1353-1917
1.7820635678284764


In [ ]:
def algorithmic_script(obj,z):
    print '%s'%(obj)
    (amp_Hb1,amp_Hb2,vel_Hb1,vel_Hb2,vel_sigma_Hb1,vel_sigma_Hb2,amp_OIII5007,vel_OIII,vel_sigma_OIII,amp_OIII5007_br,vel_OIII_br,vel_sigma_OIII_br,vel_offset,m,c) = par(obj)
    (amp_Hb_fit,amp_OIII5007_fit,vel_OIII_fit,vel_sigma_OIII_fit,amp_Hb_br_fit,amp_OIII5007_br_fit,vel_OIII_br_fit,vel_sigma_OIII_br_fit,amp_Hb1_fit,amp_Fe5018_1_fit,vel_Hb1_fit,vel_sigma_Hb1_fit,amp_Hb2_fit,amp_Fe5018_2_fit,vel_Hb2_fit,vel_sigma_Hb2_fit,m_fit,c_fit) = loadtab('%s_non_spectro_central_fit.fits'%(obj))                          
    (amp_Hb_spec,amp_OIII5007_spec,amp_OIII5007_br_spec,amp_Hb_br_spec,amp_Hb1_spec,amp_Hb2_spec,amp_Fe5018_1_spec,amp_Fe5018_2_spec,m_spec,c_spec) = loadspectab('%s_spectro_central_fit.fits'%(obj))  
    (OIII_br_data) = loadwing('maps_%s.fits'%(obj))
    
    sigma_spectro_OIII_5007_br = line_width_recons(vel_sigma_OIII_br,rest_line=5006.8,inst_res_fwhm=0)
    sigma_spectro_OIII_4959_br = line_width_recons(vel_sigma_OIII_br,rest_line=4958.9,inst_res_fwhm=0) 
    
    sigma_nonspec_OIII_5007_br = line_width_recons(vel_sigma_OIII_br_fit,rest_line=5006.8,inst_res_fwhm=0)
    sigma_nonspec_OIII_4959_br = line_width_recons(vel_sigma_OIII_br_fit,rest_line=4958.9,inst_res_fwhm=0)
    
    
    flux_wing_moffat = flux_OIII_br_moffat(OIII_br_data,sigma_spectro_OIII_5007_br,sigma_spectro_OIII_4959_br)
    flux_wing_spectro = flux_OIII_br_spectro(amp_OIII5007_br_spec,sigma_spectro_OIII_5007_br,sigma_spectro_OIII_4959_br)
    flux_wing_nonspec = flux_OIII_br_nonspec(amp_OIII5007_br_fit,sigma_nonspec_OIII_5007_br,sigma_nonspec_OIII_4959_br)
    print flux_wing_moffat,flux_wing_spectro,flux_wing_nonspec
    
    lum_wing_moffat = lum_OIII_br_moffat(z,flux_wing_moffat)
    lum_wing_spectro = lum_OIII_br_spectro(z,flux_wing_spectro)
    lum_wing_nonspec = lum_OIII_br_nonspec(z,flux_wing_nonspec)
    
    print lum_wing_moffat,lum_wing_spectro,lum_wing_nonspec 
    
    par_tab(obj,flux_wing_moffat,flux_wing_spectro,flux_wing_nonspec,lum_wing_moffat,lum_wing_spectro,lum_wing_nonspec)